<a href="https://colab.research.google.com/github/PhuongAnh2212/CS313DeepLearningforAI/blob/main/Group_Assignment_Anomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://raw.githubusercontent.com/openvinotoolkit/anomalib/main/docs/source/_static/images/logos/anomalib-wide-blue.png" alt="Paris" class="center"></center>

<center>💙 A library for benchmarking, developing and deploying deep learning anomaly detection algorithms</center>

---

> NOTE:
> This notebook is originally created by @innat on [Kaggle](https://www.kaggle.com/code/ipythonx/mvtec-ad-anomaly-detection-with-anomalib-library/notebook).

[Anomalib](https://github.com/openvinotoolkit/anomalib): Anomalib is a deep learning library that aims to collect state-of-the-art anomaly detection algorithms for benchmarking on both public and private datasets. Anomalib provides several ready-to-use implementations of anomaly detection algorithms described in the recent literature, as well as a set of tools that facilitate the development and implementation of custom models. The library has a strong focus on image-based anomaly detection, where the goal of the algorithm is to identify anomalous images, or anomalous pixel regions within images in a dataset.

The library supports a number of image and video datasets for **benchmarking** and custom dataset support for **training/inference**. In this notebook, we will explore `anomalib` training a PADIM model on the `MVTec AD` bottle dataset and evaluating the model's performance.


## Installing Anomalib


Installation can be done in two ways: (i) install via PyPI, or (ii) installing from sourc, both of which are shown below:


### I. Install via PyPI


In [1]:
# Option - I: Uncomment the next line if you want to install via pip.
# %pip install anomalib
# %anomalib install -v

> NOTE:
>
> Although v1.0.0 is on PyPI, it may not be stable and may have bugs. It is therefore recommended to install from source.


### II. Install from Source

This option would initially download anomalib repository from github and manually install `anomalib` from source, which is shown below:


In [2]:
# Option - II: Uncomment the next three lines if you want to install from the source.
!git clone https://github.com/openvinotoolkit/anomalib.git
%cd anomalib
!pip install anomalib[full]
# %anomalib install -v

fatal: destination path 'anomalib' already exists and is not an empty directory.
/content/anomalib


Now let's verify the working directory. This is to access the datasets and configs when the notebook is run from different platforms such as local or Google Colab.


In [3]:
import os
from pathlib import Path

from git.repo import Repo

current_directory = Path.cwd()
if current_directory.name == "000_getting_started":
    # On the assumption that, the notebook is located in
    #   ~/anomalib/notebooks/000_getting_started/
    root_directory = current_directory.parent.parent
elif current_directory.name == "anomalib":
    # This means that the notebook is run from the main anomalib directory.
    root_directory = current_directory
else:
    # Otherwise, we'll need to clone the anomalib repo to the `current_directory`
    repo = Repo.clone_from(
        url="https://github.com/openvinotoolkit/anomalib.git",
        to_path=current_directory,
    )
    root_directory = current_directory / "anomalib"

os.chdir(root_directory)

In [4]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage

from anomalib import TaskType
# from anomalib.data import MVTec
from anomalib.data.utils import read_image
from anomalib.deploy import ExportType, OpenVINOInferencer
from anomalib.engine import Engine
from anomalib.models import EfficientAd


/usr/local/lib/python3.10/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


## Imports


## Model

Currently, there are **13** anomaly detection models available in `anomalib` library. Namely,

- [CFA](https://arxiv.org/abs/2206.04325)
- [CS-Flow](https://arxiv.org/abs/2110.02855v1)
- [CFlow](https://arxiv.org/pdf/2107.12571v1.pdf)
- [DFKDE](https://github.com/openvinotoolkit/anomalib/tree/main/anomalib/models/dfkde)
- [DFM](https://arxiv.org/pdf/1909.11786.pdf)
- [DRAEM](https://arxiv.org/abs/2108.07610)
- [FastFlow](https://arxiv.org/abs/2111.07677)
- [Ganomaly](https://arxiv.org/abs/1805.06725)
- [Padim](https://arxiv.org/pdf/2011.08785.pdf)
- [Patchcore](https://arxiv.org/pdf/2106.08265.pdf)
- [Reverse Distillation](https://arxiv.org/abs/2201.10703)
- [R-KDE](https://ieeexplore.ieee.org/document/8999287)
- [STFPM](https://arxiv.org/pdf/2103.04257.pdf)

In this tutorial, we'll be using [Padim](https://arxiv.org/pdf/2011.08785.pdf).


## Dataset: MVTec AD

**MVTec AD** is a dataset for benchmarking anomaly detection methods with a focus on industrial inspection. It contains over **5000** high-resolution images divided into **15** different object and texture categories. Each category comprises a set of defect-free training images and a test set of images with various kinds of defects as well as images without defects. If the dataset is not located in the root datasets directory, anomalib will automatically install the dataset.

We could now import the MVtec AD dataset using its specific datamodule implemented in anomalib.


In [5]:
!pip install gdown


In [6]:
import gdown

file_id = '1SGnUk_qnCiarD0zGi7lsOSaNUKIyO7rq'

# Construct the Google Drive URL for downloading
url = f'https://drive.google.com/uc?id={file_id}'

# Download the zip file
gdown.download(url, 'dataset.tar.xz', quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=1SGnUk_qnCiarD0zGi7lsOSaNUKIyO7rq
From (redirected): https://drive.google.com/uc?id=1SGnUk_qnCiarD0zGi7lsOSaNUKIyO7rq&confirm=t&uuid=7e81f77b-db10-4a19-ae24-be5e9ff10449
To: /content/anomalib/dataset.tar.xz
100%|██████████| 156M/156M [00:03<00:00, 45.1MB/s]


'dataset.tar.xz'

In [7]:
# Create the target directory where the dataset will be extracted
import os
os.makedirs('/content/dataset', exist_ok=True)

# Now extract the .tar.xz file to the /content/dataset/ directory
!tar -xJf dataset.tar.xz -C /content/dataset/

# Verify the extracted contents
!ls /content/dataset/


bottle


In [8]:
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00


In [9]:
from google.colab import drive
from anomalib import TaskType
from anomalib.models import EfficientAd
from anomalib.engine import Engine
from anomalib.deploy import ExportType
from anomalib.callbacks import ModelCheckpoint
from anomalib.data import Folder
from loguru import logger
import os

def train():
    # Path to your extracted 'bottle' folder in Colab
    dataset_root = '/content/dataset/bottle'

    logger.info(f"Dataset root: {dataset_root}")
    normal_dir = os.path.join(dataset_root, 'train')  # Normal images (train/good)
    abnormal_dir = os.path.join(dataset_root, 'test')  # Abnormal images (test)

    # Check if the directories exist
    if not os.path.exists(normal_dir):
        logger.error(f"Normal directory does not exist: {normal_dir}")
        return
    if not os.path.exists(abnormal_dir):
        logger.error(f"Abnormal directory does not exist: {abnormal_dir}")
        return

    # Define the paths for checkpoints and model weights
    checkpoint_dir = '/content/anomaly_detection/checkpoints'
    model_weight_dir = '/content/anomaly_detection/anomalib_weight'

    # Create directories for checkpoints and model weights if they don't exist
    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(model_weight_dir, exist_ok=True)

    # Create the datamodule
    datamodule = Folder(
        name="Bottle Dataset",
        root=dataset_root,
        normal_dir=normal_dir,
        abnormal_dir=abnormal_dir,
        num_workers=8,
        task=TaskType.CLASSIFICATION,
        train_batch_size=1
    )
    datamodule.setup()

    # Initialize the EfficientAd model
    model = EfficientAd()

    # Set up the Engine with a checkpoint callback
    engine = Engine(
        max_epochs=100,
        task=TaskType.CLASSIFICATION,
        callbacks=[ModelCheckpoint(dirpath=checkpoint_dir, every_n_epochs=50, save_last=True)]
    )

    # Train the model
    logger.info("Training started...")
    engine.fit(datamodule=datamodule, model=model)

    # Export trained weights (optional)
    logger.info("Exporting model weights...")
    engine.export(export_type=ExportType.OPENVINO, model=model, export_root=model_weight_dir)

if __name__ == "__main__":
    train()


2024-11-16 16:38:47.775 | INFO     | __main__:train:15 - Dataset root: /content/dataset/bottle
2024-11-16 16:38:48.105 | INFO     | __main__:train:58 - Training started...
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation mig

Training: |          | 0/? [00:00<?, ?it/s]


efficientad_pretrained_weights.zip: 0.00B [00:00, ?B/s]
efficientad_pretrained_weights.zip:   0%|          | 8.19k/40.0M [00:00<54:30, 12.2kB/s]
efficientad_pretrained_weights.zip:  26%|██▋       | 10.5M/40.0M [00:00<00:01, 17.3MB/s]
efficientad_pretrained_weights.zip:  53%|█████▎    | 21.0M/40.0M [00:00<00:00, 31.1MB/s]
efficientad_pretrained_weights.zip:  69%|██████▉   | 27.7M/40.0M [00:01<00:00, 38.4MB/s]
efficientad_pretrained_weights.zip: 40.0MB [00:01, 32.5MB/s]                            
/usr/local/lib/python3.10/dist-packages/anomalib/models/image/efficient_ad/lightning_model.py:98: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be fli

Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 2/2 [00:04<00:00,  2.37s/it]


...

Calculate Validation Dataset Quantiles: 100%|██████████| 2/2 [00:07<00:00,  3.57s/it]


Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 2/2 [00:04<00:00,  2.30s/it]


Validation: |          | 0/? [00:00<?, ?it/s]

Calculate Validation Dataset Quantiles: 100%|██████████| 2/2 [00:07<00:00,  4.00s/it]
INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
2024-11-16 17:43:52.591 | INFO     | __main__:train:62 - Exporting model weights...
/usr/local/lib/python3.10/dist-packages/anomalib/models/image/efficient_ad/torch_model.py:28: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mean = torch.tensor([0.485, 0.456, 0.406])[None, :, None, None].to(x.device)
/usr/local/lib/python3.10/dist-packages/anomalib/models/image/efficient_ad/torch_model.py:29: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning 

In [11]:
import os
import cv2
from anomalib import TaskType
from anomalib.deploy import OpenVINOInferencer
from loguru import logger

def inference():
    # Define paths
    model_path = '/content/anomaly_detection/anomalib_weight/weights/openvino/model.bin'  # Path to OpenVINO model
    metadata_path = '/content/anomaly_detection/anomalib_weight/weights/openvino/metadata.json'  # Path to metadata
    test_images_dir = '/content/dataset/bottle/test/broken_large'  # Path to test images
    output_dir = '/content/dataset/bottle/test/output'  # Folder where you want to save the output images

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Initialize the inferencer
    inferencer = OpenVINOInferencer(
        path=model_path,
        metadata=metadata_path,
        device="CPU",
        task=TaskType.CLASSIFICATION
    )
    logger.info('Model loaded successfully.')

    # Iterate over test images and make predictions
    for img_name in os.listdir(test_images_dir):
        img_path = os.path.join(test_images_dir, img_name)
        if os.path.isfile(img_path):  # Ensure it's a file
            # Predict the anomaly
            predictions = inferencer.predict(img_path)

            # Log the prediction
            logger.info(f"{img_name}: {predictions.pred_label}, {predictions.pred_score:.4f}")

            # Get the segmented image with the highlighted anomaly regions
            segmented_img = predictions.segmentations  # This should be the output image with the detected region

            # Save the segmented image to the output directory
            output_img_path = os.path.join(output_dir, f"segmented_{img_name}")
            cv2.imwrite(output_img_path, segmented_img)  # Save the image

if __name__ == "__main__":
    inference()


2024-11-16 17:43:57.180 | INFO     | __main__:inference:24 - Model loaded successfully.
2024-11-16 17:43:58.422 | INFO     | __main__:inference:34 - 013.png: 1, 0.5089
2024-11-16 17:43:59.392 | INFO     | __main__:inference:34 - 017.png: 1, 0.5321
2024-11-16 17:44:00.527 | INFO     | __main__:inference:34 - 014.png: 1, 0.5178
2024-11-16 17:44:01.890 | INFO     | __main__:inference:34 - 012.png: 1, 0.5164
2024-11-16 17:44:04.470 | INFO     | __main__:inference:34 - 007.png: 1, 0.5096
2024-11-16 17:44:07.261 | INFO     | __main__:inference:34 - 015.png: 1, 0.5136
2024-11-16 17:44:08.904 | INFO     | __main__:inference:34 - 000.png: 1, 0.5110
2024-11-16 17:44:10.290 | INFO     | __main__:inference:34 - 001.png: 1, 0.5179
2024-11-16 17:44:11.996 | INFO     | __main__:inference:34 - 005.png: 1, 0.5110
2024-11-16 17:44:13.445 | INFO     | __main__:inference:34 - 010.png: 1, 0.5106
2024-11-16 17:44:14.776 | INFO     | __main__:inference:34 - 019.png: 1, 0.5165
2024-11-16 17:44:15.540 | INFO  

### YAYYYYY IT SLAYED
From QA and PA with love <3

In [22]:
!cp model.xml /content/anomaly_detection/anomalib_weight/weights/openvino
!cp model.bin /content/anomaly_detection/a!zip -r /content/model.zip /content/model/
nomalib_weight/weights/openvino

cp: cannot stat 'path_to_model.xml': No such file or directory
cp: cannot stat 'path_to_model.bin': No such file or directory


In [ ]:
pip freeze > requirements.txt